**Содержание**<a id='toc0_'></a>    
- [Выбор локации для скважины](#toc1_)    
  - [Загрузка и подготовка данных](#toc1_1_)    
    - [Инфо](#toc1_1_1_)    
    - [Пропуски](#toc1_1_2_)    
    - [Дубликаты](#toc1_1_3_)    
    - [Делим на выборки](#toc1_1_4_)    
  - [Обучение и проверка модели](#toc1_2_)    
  - [Подготовка к расчёту прибыли](#toc1_3_)    
    - [Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.](#toc1_3_1_)    
    - [Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели](#toc1_3_2_)    
  - [Расчёт прибыли и рисков](#toc1_4_)    
    - [Вывод](#toc1_4_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Выбор локации для скважины](#toc0_)

Описание проекта:

Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## <a id='toc1_1_'></a>[Загрузка и подготовка данных](#toc0_)

### <a id='toc1_1_1_'></a>[Инфо](#toc0_)

In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
from numpy.random import RandomState


In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
print(data_0.info())
print()
print('Первые 5 строк датасета:')
print(data_0.head())
print()
print('Стат.показатели:')
print(data_0.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Первые 5 строк датасета:
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

Стат.показатели:
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0

In [3]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
print(data_1.info())
print()
print('Первые 5 строк датасета:')
print(data_1.head())
print()
print('Стат.показатели:')
print(data_1.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Первые 5 строк датасета:
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305

Стат.показатели:
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
s

In [4]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
print(data_2.info())
print()
print('Первые 5 строк датасета:')
print(data_2.head())
print()
print('Стат.показатели:')
print(data_2.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Первые 5 строк датасета:
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746

Стат.показатели:
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1

Мы имеем 3 датасета по 100.000 строк каждый.

В датасетах по 5 столбцов:

    - id - уникальный идентификатор скважины
    - f0 - признак точки
    - f1 - признак точки
    - f2 - признак точки
    - product - объём запасов в скважине (тыс. баррелей)

### <a id='toc1_1_2_'></a>[Пропуски](#toc0_)

In [5]:
data_full = ['data_0','data_1','data_2']
for data_n in data_full:
    data_check = globals()[data_n]  # получаем датасет по его имени
    data_miss = data_check.isnull().sum()  # считаем количество пропусков в датасете
    print(f"Количество пропусков в датасете {data_n}: {data_miss}")
    print()
    print('-------------------')

Количество пропусков в датасете data_0: id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

-------------------
Количество пропусков в датасете data_1: id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

-------------------
Количество пропусков в датасете data_2: id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

-------------------


Пропусков в датасете не обнаружено. 

### <a id='toc1_1_3_'></a>[Дубликаты](#toc0_)

In [6]:
data_full= [data_0, data_1,data_2]

# Цикл for для проверки каждого столбца каждого датасета
for df in data_full:  # Перебираем каждый датасет
    print("Количество дубликатов по каждому столбцу:")
    print()
    for column in df.columns:  # Перебираем каждый столбец датасета
        duplicates = df[column].duplicated().sum()  # Считаем количество дубликатов в столбце
        print(column + ": " + str(duplicates))  # Выводим количество дубликатов для столбца
    print("\n")

Количество дубликатов по каждому столбцу:

id: 10
f0: 0
f1: 0
f2: 0
product: 0


Количество дубликатов по каждому столбцу:

id: 4
f0: 0
f1: 0
f2: 0
product: 99988


Количество дубликатов по каждому столбцу:

id: 4
f0: 0
f1: 0
f2: 0
product: 0




Мы видим, что у нас есть дубликаты в колонке id во всех трех датасетах и в колонке product во втором датасете. 

Что мы можем сделать:

    1. Колонку projuct мы не трограем, так как там указываются объемы запаса скважин, такие данные могут дублироваться. 
    2. Дубли в колонке id - можно удалить, так как это уникальный идентификатор, дублирование такой строки в большинстве случаев говорит о задвоении данных. К тому же таких строк не много и их удаление не повлияет на дальнейшее исследование. 
    3. Не трогать дубли в колонке id, так как при дальнейшей работе мы отбросим этот столбец, как "неимформативный признак". И возвращаясь, к мысли со 2 пункта, там всего 18 строк. 
    
Я предлагаю остановиться на 3 варианте, а имеено не ликвидировать дубли, а просто отбросить столбец.

In [7]:
for df in data_full:
    df.drop('id', axis=1, inplace=True)

### <a id='toc1_1_4_'></a>[Делим на выборки](#toc0_)

In [8]:
#d0
target = data_0['product']
features = data_0.drop('product', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [9]:
#check
display(features_train.shape)
display(features_valid.shape)

(75000, 3)

(25000, 3)

In [10]:
#d1
target_1 = data_1['product']
features_1 = data_1.drop('product', axis=1)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

In [11]:
#check
display(features_train_1.shape)
display(features_valid_1.shape)

(75000, 3)

(25000, 3)

In [12]:
#d2
target_2 = data_2['product']
features_2 = data_2.drop('product', axis=1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [13]:
#check
display(features_train_2.shape)
display(features_valid_2.shape)

(75000, 3)

(25000, 3)

## <a id='toc1_2_'></a>[Обучение и проверка модели](#toc0_)

In [14]:
#d0 linear_regression
model = LinearRegression() 
model.fit(features_train, target_train) # обучили модель на тренировочной выборке
predicted_valid = model.predict(features_valid) # получили предсказания модели на валидационной выборке
r2= r2_score(target_valid, predicted_valid)
result =mean_squared_error(target_valid, predicted_valid)**0.5   # посчитали значение метрики RMSE на валидационной выборке
print("RMSE модели на валидационной выборке:", result)
print('Средний запас сырья:',predicted_valid.mean())
print("R2 =", r2)

RMSE модели на валидационной выборке: 37.5794217150813
Средний запас сырья: 92.59256778438035
R2 = 0.27994321524487786


In [15]:
#d1 linear_regression
model = LinearRegression() 
model.fit(features_train_1, target_train_1) # обучили модель на тренировочной выборке
predicted_valid_1 = model.predict(features_valid_1) # получили предсказания модели на валидационной выборке
r2= r2_score(target_valid_1, predicted_valid_1)
result =mean_squared_error(target_valid_1, predicted_valid_1)**0.5   # посчитали значение метрики RMSE на валидационной выборке
print("RMSE модели на валидационной выборке:", result)
print('Средний запас сырья:',predicted_valid_1.mean())
print("R2 =", r2)

RMSE модели на валидационной выборке: 0.893099286775617
Средний запас сырья: 68.728546895446
R2 = 0.9996233978805127


In [16]:
#d2 linear_regression
model = LinearRegression() 
model.fit(features_train_2, target_train_2) # обучили модель на тренировочной выборке
predicted_valid_2 = model.predict(features_valid_2) # получили предсказания модели на валидационной выборке
r2= r2_score(target_valid_2, predicted_valid_2)
result =mean_squared_error(target_valid_2, predicted_valid_2)**0.5   # посчитали значение метрики RMSE на валидационной выборке
print("RMSE модели на валидационной выборке:", result)
print('Средний запас сырья:',predicted_valid_2.mean())
print("R2 =", r2)

RMSE модели на валидационной выборке: 37.5794217150813
Средний запас сырья: 92.59256778438035
R2 = 0.27994321524487786


Показетели первой и третей модели хуже, чем второй. Они выигрывают по среднему заполнению, но показатель модели R2 говорит нам о том, что они весьма плохо отображают действительность, в отличии от модели под номером 2. Ее предсказания можно посчитать идеальными, так как ее значение 0.99. 

RMSE второй модели 0.89

## <a id='toc1_3_'></a>[Подготовка к расчёту прибыли](#toc0_)

Условия задачи:

- [x] Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- [x] При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- [x] Бюджет на разработку скважин в регионе — 10 млрд рублей.
- [x] При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- [x] После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [17]:
points = 500 # точки
best_points = 200 # лучшие точки для разработки
budget = 10000000000 # бюджет
barrel = 450 # цена за 1 баррель в рублях
barrel_unit = 1000 # кол-во продукта
icome_barrel = barrel*barrel_unit # Доход с каждой единицы продукта




### <a id='toc1_3_1_'></a>[Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.](#toc0_)


In [18]:
min_new_points= budget / best_points / icome_barrel
print(f'Минимальный объем сырья для безубыточной точки: {min_new_points}')

Минимальный объем сырья для безубыточной точки: 111.11111111111111


In [19]:
print('Первый регион:')
print(f'Средний объем сырья в регионе:', data_0['product'].mean())
print('Второй регион:')
print(f'Средний объем сырья в регионе:', data_1['product'].mean())
print('Третий регион:')
print(f'Средний объем сырья в регионе:', data_2['product'].mean())

Первый регион:
Средний объем сырья в регионе: 92.50000000000001
Второй регион:
Средний объем сырья в регионе: 68.82500000000002
Третий регион:
Средний объем сырья в регионе: 95.00000000000004


Не смотря на то, что средние объемы в регионах отображают цифры меньше минимальной планки. Стоит отметить, что в регионах есть множество точек с околонулевым или же нулевым значением, из-за этого средний показатель по региону снизился. 

### <a id='toc1_3_2_'></a>[Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели](#toc0_)

In [20]:
def calc_income(predictions, target):
    predictions = pd.Series(predictions).reset_index(drop=True)
    target = pd.Series(target).reset_index(drop=True)
    # Сортировка предсказаний по убыванию
    sorted_predictions = pd.Series(predictions).sort_values(ascending=False)
    
    # Получение первых 200 строк отсортированных предсказаний
    top_200_predicted = sorted_predictions[:200]
    
    # Получение соответствующих запасов сырья из массива целевых значений, используя индексы отсортированных предсказаний
    top_200_target = target[top_200_predicted.index]
    
    # Расчет итогового дохода
    
    total_income = top_200_target.sum() * icome_barrel - budget
    return total_income

In [21]:
calc_income(predicted_valid, target_valid)

3320826043.1398506

In [22]:
calc_income(predicted_valid_1, target_valid_1)

2415086696.681511

In [23]:
calc_income(predicted_valid_2, target_valid_2)

3320826043.1398506

## <a id='toc1_4_'></a>[Расчёт прибыли и рисков](#toc0_)

In [24]:
state = np.random.RandomState(12345)
values = []
predicted = pd.Series(predicted_valid).reset_index(drop=True)  
target = pd.Series(target_valid).reset_index(drop=True) 
for i in range(1000):
    pred_sample = predicted.sample(n=points, replace=True, random_state=state) 
    target_sample = target.loc[pred_sample.index]  
    values.append(calc_income(pred_sample, target_sample))  


values = pd.Series(values)
values_mean = values.mean()
lover = values.quantile(0.025)
upper = values.quantile(0.975)

print('Первый регион: \n'
      f'Минимальная прибыль: {lover:,.2f} руб.\n'
      f'Максимальаня прибыль: {upper:,.2f} руб.\n'
      f'Среднее значение: {values.mean():,.2f} руб.\n'
      f'Риск получить убытки: {values[(values) < 0].size / values.size:.2%}')

Первый регион: 
Минимальная прибыль: -111,215,545.89 руб.
Максимальаня прибыль: 909,766,941.55 руб.
Среднее значение: 396,164,984.80 руб.
Риск получить убытки: 6.90%


In [25]:
values = []
predicted = pd.Series(predicted_valid_1).reset_index(drop=True)  
target = pd.Series(target_valid_1).reset_index(drop=True) 
for i in range(1000):
    pred_sample = predicted.sample(n=points, replace=True, random_state=state) 
    target_sample = target.loc[pred_sample.index]  
    values.append(calc_income(pred_sample, target_sample))  


values = pd.Series(values)
values_mean = values.mean()
lover = values.quantile(0.025)
upper = values.quantile(0.975)

print('Второй регион: \n'
      f'Минимальная прибыль: {lover:,.2f} руб.\n'
      f'Максимальаня прибыль: {upper:,.2f} руб.\n'
      f'Среднее значение: {values.mean():,.2f} руб.\n'
      f'Риск получить убытки: {values[(values) < 0].size / values.size:.2%}')

Второй регион: 
Минимальная прибыль: 78,050,810.75 руб.
Максимальаня прибыль: 862,952,060.26 руб.
Среднее значение: 461,155,817.28 руб.
Риск получить убытки: 0.70%


In [26]:
values = []
predicted = pd.Series(predicted_valid_2).reset_index(drop=True)  
target = pd.Series(target_valid_2).reset_index(drop=True) 
for i in range(1000):
    pred_sample = predicted.sample(n=points, replace=True, random_state=state) 
    target_sample = target.loc[pred_sample.index]  
    values.append(calc_income(pred_sample, target_sample))  


values = pd.Series(values)
values_mean = values.mean()
lover = values.quantile(0.025)
upper = values.quantile(0.975)

print('Третий регион: \n'
      f'Минимальная прибыль: {lover:,.2f} руб.\n'
      f'Максимальаня прибыль: {upper:,.2f} руб.\n'
      f'Среднее значение: {values.mean():,.2f} руб.\n'
      f'Риск получить убытки: {values[(values) < 0].size / values.size:.2%}')

Третий регион: 
Минимальная прибыль: -121,346,142.82 руб.
Максимальаня прибыль: 954,523,567.68 руб.
Среднее значение: 405,860,055.03 руб.
Риск получить убытки: 5.90%


Самым выгодным регионом для создания новых точек является второй регион, тут и самый минимальный риск получить убытки и показатель среднего выше чем в других регионах. 

### <a id='toc1_4_1_'></a>[Вывод](#toc0_)

- Были получены и изучены 3 датасета (3 региона) для разработки. 
- Проверили на дубликаты, были обнаружены дубликаты в по столбцу id в количестве 18 строк на все 3 датасета. 
- некоторые дублирущие строки дублировались только по id, по остальным значениям были различия. 
- было принято решение не удалять дубликаты, их очень мало и на исследование негативно они не повлияют, к тому же, не все дубликаты были точным копией строки. 
- пропусков в датасете не обнаружено
- убрали не информативный столбец id, он не нужен нам для исследования.
- поделили каждый датасет на выборки
- Сделали и обучили модели на каждый регион. 
- Модель для второго региона показала лучший результат R2 = 0.99
- произвели подготовку к расчету прибыли:

    - объявили константы
    - рассчитали минимальный объем сырья для безубыточной точки: 111.11
    - рассчитали средний объем сырья для каждого региона
        - Не смотря что показатели в регионах оказались нижи минимального порога, не стоит забывать, что в регионах есть множество точек с околонулевым или же нулевым значением, из-за этого средний показатель по региону снизился.
    - Написали функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели
- Произвели рассчет прибыли и рисков:
    - лучший регион по разработке является регион под номером 2, так как у него самый минимальный процент риска убытков 0.7
    - также выше среднее значение прибыли с разницей в ~68 млн рублей. 